In [99]:
from Bio.PDB import PDBParser
import numpy as np
import pandas as pd

# 필요한 함수 및 변수 정의
def calc_distance(coord1, coord2):
    return np.linalg.norm(coord1 - coord2)

def is_contact(atom, ligand_atoms, cutoff_distance=5.0):
    for ligand_atom in ligand_atoms:
        if calc_distance(atom.coord, ligand_atom.coord) <= cutoff_distance:
            return True
    return False

feature_map = {
    "contact": 0,
    "backbone": 1,
    "sidechain": 2,
    "polar": 3,
    "hydrophobic": 4,
    "acceptor": 5,
    "donor": 6,
    "aromatic": 7,
    "charged": 8
}

polar_residues = {"ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "HIS", "LYS", "SER", "THR", "TYR"}
hydrophobic_residues = {"ALA", "ILE", "LEU", "MET", "PHE", "PRO", "TRP", "VAL"}
aromatic_residues = {"PHE", "TYR", "TRP", "HIS"}
charged_residues = {"ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "HIS", "LYS"}

# PDB 파일 로드
protein_pdb_file = "Recepter.pdb"
ligand_pdb_file = "ligand.pdb"

parser = PDBParser()
protein_structure = parser.get_structure("protein", protein_pdb_file)
ligand_structure = parser.get_structure("ligand", ligand_pdb_file)

# 단백질 및 리간드 원자 추출
protein_atoms = [atom for atom in protein_structure.get_atoms()]
ligand_atoms = [atom for atom in ligand_structure.get_atoms()]

# 원-핫 인코딩 벡터 초기화
one_hot_vectors = []



# 단백질 아미노산 잔기별 상호작용 확인 및 원-핫 인코딩
for residue in protein_structure.get_residues():
    if residue.id[0] == " ":
        features = set()

        # 특징 계산
        for atom in residue.get_atoms():
            if is_contact(atom, ligand_atoms):
                features.add("contact")

                if atom.name in ["N", "CA", "C", "O"]:
                    features.add("backbone")
                else:
                    features.add("sidechain")

        if residue.resname in polar_residues:
            features.add("polar")
        if residue.resname in hydrophobic_residues:
            features.add("hydrophobic")
        if residue.resname in aromatic_residues:
            features.add("aromatic")
        if residue.resname in charged_residues:
            features.add("charged")
        
        # H-bond donor/acceptor 
        if residue.resname in ["ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "HIS", "LYS", "SER", "THR", "TYR"]:
            if "H" in atom.name:
                features.add("donor")
            elif atom.name in ["O", "N"]:
                features.add("acceptor")

        # 원-핫 인코딩 벡터 생성
        one_hot_vector = [0] * len(feature_map)
        for feature in features:
            if feature in feature_map:
                one_hot_vector[feature_map[feature]] = 1

        one_hot_vectors.append(one_hot_vector)
        
        
# 결과 출력
for residue, one_hot_vector in zip(protein_structure.get_residues(), one_hot_vectors):
    if residue.id[0] == " ":
        print(f"Residue: {residue.resname} {residue.id[1]}\nOne-hot vector: {one_hot_vector}\n")       
#
# 원-핫 인코딩 벡터를 처리하여, 첫 번째 값이 0인 경우 모든 값을 0으로 변경하고, 아닌 경우 그대로 유지한 뒤, 이를 하나의 1차원 배열로 만듭니다.
result = np.array([vec if vec[0] != 0 else [0] * len(vec) for vec in one_hot_vectors]).flatten()

Residue: PHE 55
One-hot vector: [0, 0, 0, 0, 1, 0, 0, 1, 0]

Residue: LEU 56
One-hot vector: [0, 0, 0, 0, 1, 0, 0, 0, 0]

Residue: SER 57
One-hot vector: [0, 0, 0, 1, 0, 0, 1, 0, 0]

Residue: LEU 58
One-hot vector: [0, 0, 0, 0, 1, 0, 0, 0, 0]

Residue: ASP 59
One-hot vector: [0, 0, 0, 1, 0, 0, 1, 0, 1]

Residue: SER 60
One-hot vector: [0, 0, 0, 1, 0, 0, 1, 0, 0]

Residue: PRO 61
One-hot vector: [0, 0, 0, 0, 1, 0, 0, 0, 0]

Residue: THR 62
One-hot vector: [0, 0, 0, 1, 0, 0, 1, 0, 0]

Residue: TYR 63
One-hot vector: [0, 0, 0, 1, 0, 0, 1, 1, 0]

Residue: VAL 64
One-hot vector: [0, 0, 0, 0, 1, 0, 0, 0, 0]

Residue: LEU 65
One-hot vector: [0, 0, 0, 0, 1, 0, 0, 0, 0]

Residue: TYR 66
One-hot vector: [0, 0, 0, 1, 0, 0, 1, 1, 0]

Residue: ARG 67
One-hot vector: [0, 0, 0, 1, 0, 0, 1, 0, 1]

Residue: ASP 68
One-hot vector: [0, 0, 0, 1, 0, 0, 1, 0, 1]

Residue: ARG 69
One-hot vector: [0, 0, 0, 1, 0, 0, 1, 0, 1]

Residue: ALA 70
One-hot vector: [0, 0, 0, 0, 1, 0, 0, 0, 0]

Residue: GLU 71
One-hot 

In [107]:
import pandas as pd

# 데이터프레임 초기화
df = pd.DataFrame()

# 아미노산 잔기별로 원-핫 인코딩 벡터를 생성하여 데이터프레임에 추가
for residue in protein_structure.get_residues():
    if residue.id[0] == " ":
        residue_id = residue.id[1]
        features = set()

        # 특징 계산
        for atom in residue.get_atoms():
            if is_contact(atom, ligand_atoms):
                features.add("contact")

                if atom.name in ["N", "CA", "C", "O"]:
                    features.add("backbone")
                else:
                    features.add("sidechain")

        if residue.resname in polar_residues:
            features.add("polar")
        if residue.resname in hydrophobic_residues:
            features.add("hydrophobic")
        if residue.resname in aromatic_residues:
            features.add("aromatic")
        if residue.resname in charged_residues:
            features.add("charged")
        
        # H-bond donor/acceptor 판별은 아래와 같이 진행할 수 있지만, 더 정확한 방법은 PDB 파일로부터 각 원자의 전하 상태를 확인하는 것입니다.
        # 이는 복잡한 작업이므로, 도움이 필요한 경우 추가 자료를 참조하시기 바랍니다.
        if residue.resname in ["ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "HIS", "LYS", "SER", "THR", "TYR"]:
            for atom in residue.get_atoms():
                if "H" in atom.name:
                    features.add("donor")
                elif atom.name in ["O", "N"]:
                    features.add("acceptor")

        # 원-핫 인코딩 벡터 생성
        one_hot_vector = [0] * len(feature_map)
        for feature in features:
            if feature in feature_map:
                one_hot_vector[feature_map[feature]] = 1

        # 컬럼 이름 생성
        column_names = [f"{residue_id}_{k}" for k in feature_map.keys()]

        # 데이터프레임에 추가
        df = pd.concat([df, pd.DataFrame([one_hot_vector], columns=column_names)], axis=0, ignore_index=True)


In [101]:
# 컬럼 이름에 A를 추가하여 새로운 컬럼 이름 리스트 생성
new_columns = ['A' + column for column in df.columns]

# 데이터프레임 컬럼 이름 변경
df.columns = new_columns

In [102]:
columns_list = df.columns.tolist()

In [103]:
len(columns_list)

2907

In [104]:
len(result)

2907

In [105]:
DATA_CSV = pd.DataFrame(result.reshape(1, -1), columns=columns_list)

In [106]:
DATA_CSV

,A55_contact,A55_backbone,A55_sidechain,A55_polar,A55_hydrophobic,A55_acceptor,A55_donor,A55_aromatic,A55_charged,A56_contact,...,A376_charged,A377_contact,A377_backbone,A377_sidechain,A377_polar,A377_hydrophobic,A377_acceptor,A377_donor,A377_aromatic,A377_charged
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
DATA_CSV.to_csv('output.csv', index=False)